Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [671]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import copy, deepcopy


from pprint import pprint

import numpy as np

In [672]:
PROBLEM_SIZE = 101



In [673]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15
current_state = [1,2]
NUM_ROWS = len(current_state)


In [674]:
def fitness1(state):
    ###todolater
    nim_sum = 0
    for i in range(NUM_ROWS):
        nim_sum ^= state[i]
    return -nim_sum

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

fitness = fitness1

In [675]:
@dataclass
class Individual:
    fitness: int
    genotype: list[int]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    if pool[0].genotype[1]>= current_state[pool[0].genotype[0] ] and pool[1].genotype[1]>= current_state[pool[1].genotype[0]]:
        return Individual(fitness=None, genotype=[randint(0,NUM_ROWS-1), 0])
    elif pool[0].genotype[1]>= current_state[pool[0].genotype[0]]:
        return pool[1]
    elif pool[1].genotype[1]>= current_state[pool[1].genotype[0]]:
        return pool[0]

    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    offspring.genotype[1]+=1
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

In [676]:
population = [
    Individual(
        genotype= [_, 1],
        #genotype = np.concatenate([np.zeros(_, dtype=int), [1], np.zeros(NUM_ROWS-_-1, dtype=int)]),
        fitness=None,
    )
    for _ in range(NUM_ROWS)
]

for _, i in enumerate(population):
    new_state= copy(current_state)
    new_state[i.genotype[0]]-= i.genotype[1]
    i.fitness = fitness(new_state)



In [677]:
print(population)


[Individual(fitness=-2, genotype=[0, 1]), Individual(fitness=0, genotype=[1, 1])]


In [678]:
def soluzione(population):
    for generation in range(1000):
        if(len(current_state)==2 and (current_state[0]==1 or current_state[1]==1)):
            if(current_state[0]==1):
                return Individual(fitness=None, genotype=[1,current_state[1]])
            else:
                return Individual(fitness=None, genotype=[0,current_state[0]])
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            if random() < 1:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p)
                #print("parent is {}, offspring is {}".format(p , o))

            offspring.append(o)

        for i in offspring:
            new_state= copy(current_state)
            new_state[i.genotype[0]]-= i.genotype[1]
            i.fitness = fitness(new_state) 
            #print(new_state, i) 
            
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
    return population[0]
    
    


In [679]:
print(soluzione(population))

Individual(fitness=None, genotype=[1, 2])
